## Montamos Drive

In [ ]:
# Montamos el Drive al Notebook
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Cambiamos de directorio al Drive
import os
os.chdir("drive/My Drive/Hackaton2021/codigo/2022/")
!ls

data  labels  ProcesaTexto.ipynb


## Cargamos las librerias

In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Cargamos los datos

In [ ]:
# Cargamos el texto extraído de las imágenes de los archivos de los desaparecidos
df = pd.read_csv("data/Texto_extraido.csv", header=None)

# Cargamos los archivos con la información a buscar
with open("labels/organizations.csv") as f:
    organizations = f.read()
f.close()
with open("labels/prosecuted.csv") as f:
    prosecuted = f.read()
f.close()
with open("labels/civilservants.csv") as f:
    civilservants = f.read()
f.close()
with open("labels/places.csv") as f:
    places = f.read()
f.close()

## Procesamos el texto

### Creamos bolsas de palabras por cada archivo de información

In [ ]:
# Creamos los vectores de contexto con los archivos de información a buscar
context_organizations = organizations.lower().split("\n")
context_prosecuted = prosecuted.lower().split("\n")
context_civilservants = civilservants.lower().split("\n")
context_places = places.lower().split("\n")

In [ ]:
# Tokenizamos
tokenizer = RegexpTokenizer(r'[a-zA-ZáéíóúñÁÉÍÓÚÜüÑ]{1,}')

# Creamos bolsas de palabras (unigramas) con la información a buscar
vectorizer_organizations = TfidfVectorizer(tokenizer=tokenizer.tokenize, norm='l2')
bag_of_words_organizations = vectorizer_organizations.fit_transform(context_organizations)

vectorizer_prosecuted = TfidfVectorizer(tokenizer=tokenizer.tokenize, norm='l2')
bag_of_words_prosecuted = vectorizer_prosecuted.fit_transform(context_prosecuted)

vectorizer_civilservants = TfidfVectorizer(tokenizer=tokenizer.tokenize, norm='l2')
bag_of_words_civilservants = vectorizer_civilservants.fit_transform(context_civilservants)

vectorizer_places = TfidfVectorizer(tokenizer=tokenizer.tokenize, norm='l2')
bag_of_words_places = vectorizer_places.fit_transform(context_places)

### Tokenizamos y creamos un vocabulario con todos los archivos de información

In [ ]:
# Tokenizamos
organizations = tokenizer.tokenize(organizations.lower())
prosecuted = tokenizer.tokenize(prosecuted.lower())
civilservants = tokenizer.tokenize(civilservants.lower())
places = tokenizer.tokenize(places.lower())

In [ ]:
# Creamos un vocabulario de las palabras que debemos encontrar en los documentos
vocabulary = list(set(organizations) | set(prosecuted) | set(civilservants) | set(places))
#vocabulary = [word.lower() for word in vocabulary if len(word)>2 ]
vocabulary[:10]

['chewe',
 'arroyo',
 'pisa',
 'morkor',
 'abadalia',
 'cabellos',
 'terminal',
 'orias',
 'juarense',
 'arrollo']

## Iniciamos el procesamiento de los textos segmentados

In [ ]:
# Seleccionamos el texto a procesar
text = df[1][0]

# Visualizamos el texto extraido de un archivo
print(text)

| ~~ Acapulco ~ El dia de ayer a las 20 00 horas en la
Terminal de Autobuses de la Linea *Flecha Roja”
Puerto fue detenida JPOLINAR GU”MAN BENITEZ (a

quien fuera miembro activo de las fuerzas

ri _s, ex estudiante de la Preparato la a este lugar y ex
mill ante del P rtido Comunicta Mexicano v puesta a disposi =
cibn del Tte Cor “JIRIO ARTURO ACOSTA CHAPARRO E CAPITE Jefeo-
de Policia y Trénsito }

; GUZMAN B71! I™EZ es esposa del activista Lic 8 a IW,
“La Lombriz“, quien actualmente se encuentra trabajuide,en

moresa General Motors®" de Toluca 2£& 3 en abril de 1974
en una reunidn convocada por JUAN GUILLLRMO HER! ANDEZ
Chilpancingo acept$ prepararse para constitul
Revolucionario que desarrollarfa por igual la lucha democriti
ca gue la lucha ilegal (armada) ColaborS en el peribédico clan
destino "El Topo Rojo" editado por BER! ANDEZ DIAZ con marca-
da tendencia de la Liga Comunista "23 de Septiembre” en inclu-
sive sa reprodujeron algunos articulo de “Madera

CUZMAN BENIT®? durante lo

### Corregimos el texto segmentado

In [ ]:
def correction_words(text, vocabulary, tokenizer):
    # Tokenizamos el texto
    tokens = tokenizer.tokenize(text.lower())

    # Corregimos el texto utilizando la metrica de jaccard
    tokens_corregidos = []
    for word in tokens:# [50:61]:
        if len(word)>2:
            temp = [ (jaccard_distance(set(nltk.ngrams(word, 2)), set(nltk.ngrams(w, 2))),w) 
                    for w in vocabulary]

            score, word_replace = sorted(temp, key = lambda val:val[0])[0]
            if score < 0.5:
                tokens_corregidos.append(word_replace)
                #print(word, word_replace)
    return(tokens_corregidos)

In [ ]:
# Extraemos token del texto y corregimos las palabras con el vocabulario
tokens_corregidos = correction_words(text, vocabulary, tokenizer)

# Visualizamos los tokens corregidos
print(tokens_corregidos)

['acapulco', 'dia', 'mayer', 'las', 'horas', 'terminal', 'autobuses', 'linea', 'flecha', 'roja', 'puerto', 'apolinar', 'manana', 'benitez', 'quienn', 'afuera', 'miembros', 'las', 'fuerzas', 'estudiantes', 'preparatora', 'este', 'luga', 'milla', 'ante', 'del', 'partido', 'comunista', 'mexicano', 'puesto', 'del', 'coro', 'pirio', 'arturo', 'acosta', 'chaparro', 'escapite', 'jefe', 'policia', 'transito', 'guzman', 'esposa', 'del', 'activistas', 'lic', 'ambriz', 'quienn', 'encuentro', 'mores', 'general', 'motors', 'toluca', 'abril', 'zuna', 'por', 'juan', 'guillermo', 'hera', 'sandez', 'chilpancingo', 'preparatora', 'para', 'constitucion', 'revolucionario', 'que', 'desarrollo', 'por', 'iguala', 'lucha', 'guel', 'lucha', 'ballega', 'armada', 'periodico', 'clan', 'clandestino', 'topo', 'rojo', 'por', 'uber', 'sandez', 'diaz', 'con', 'marc', 'tendencia', 'liga', 'comunista', 'septiembre', 'articulo', 'madera', 'guzman', 'benita', 'durant', 'los', 'primero', 'territorios', 'que', 'caba', 'con'

### Indentificamos el archivo al que pertenece (lugar, organización, nombre, etc) cada token

In [ ]:
def match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 1, thres=0.8, view=False):
    # Calculamos los n-gramas
    if n > 1:
        tokens_corregidos = [" ".join(words) for words in nltk.ngrams(tokens_corregidos,n)]

    # Creamos la bolsa de palabras
    bow = vectorizer.transform(tokens_corregidos)
    bow = bow.toarray() # convertimos a array
    if view:
        print(bow.shape)
        print(len(tokens_corregidos))

    match = []
    # Calculamos la similitud coseno
    for i in range(len(tokens_corregidos)):
        #i = 1
        cs = cosine_similarity(bow[i,:].reshape(1, -1), bag_of_words)

        if cs.max()> thres:
            match.append(context[np.argmax(cs)])
            if view:
                print("-----------------")
                print("token:   ", tokens_corregidos[i])
                print("similar: ", context[np.argmax(cs)])
                print("probab:  ", cs.max())
                print("index:   ", np.argmax(cs))
            
    return(match)

In [ ]:
def match_info(tokens_corregidos, vectorizer, bag_of_words, context):
    # Creamos vectores de contexto con 1-gramas, 2-gramas, 3-gramas, 4-gramas y 5-gramas
    # y matcheamos con la similitud coseno a cada bolsa de palabras de cada archivo de información
    match1 = match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 1)
    match2 = match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 2)
    match3 = match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 3)
    match4 = match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 4)
    match5 = match_bag_of_words(tokens_corregidos, vectorizer, bag_of_words, context, n = 5)

    # Unimos las palabras encontradas con los diferentes n-gramas
    match = set(match1) | set(match2) | set(match3) | set(match4) | set(match5)
    return(list(match))

In [ ]:
# Detectamos organizaciones, personas, lugares y funcionarios en el texto extraido
detect_info = {}

detect_info['organizations'] = match_info(tokens_corregidos, vectorizer_organizations, bag_of_words_organizations, context_organizations)
detect_info['prosecuted'] = match_info(tokens_corregidos, vectorizer_prosecuted, bag_of_words_prosecuted, context_prosecuted)
detect_info['civilservants'] = match_info(tokens_corregidos, vectorizer_civilservants, bag_of_words_civilservants, context_civilservants)
detect_info['places'] = match_info(tokens_corregidos, vectorizer_places, bag_of_words_places, context_places)

# Imprimimos los resultados
for key in detect_info:
    print("--- ", key, ": ")
    print(detect_info[key])

---  organizations : 
['grupo constitucion', 'grupo del 68', 'partido comunista', '22 de abril', 'periodico 30-30', 'periodico rojo', 'el mexicano', 'periodico madera', 'el dia', 'diario sol', 'partido comunista mexicano', 'flecha roja', '24 horas', 'el sol', '23 de septiembre', 'fuerzas armadas de liberacion', 'diario el sol de chilpancingo', 'comite de lucha armada', 'diario de toluca', 'escuela preparatorio de chilpancingo, chilpancingo, guerrero', 'policia', 'el sol de chilpancingo', 'liga comunista 23 de septiembre', 'sindicato revolucionario']
---  prosecuted : 
['jose rojo', 'raul sanchez dia', 'juan coro', 'vicente chaparro', 'alfonso tena', 'fernando ante', 'arturo', 'rosalio wences reza', 'mario arturo acosta chaparro escapite', 'juan guillermo hernandez', 'jose toluca', 'maria benitez', 'juan diaz diaz', 'liga comunista 23 de septiembre', 'guillermo hernandez', 'francisco mencio gonzalez', 'mario arturo acosta chaparro', 'guerrero guerrero raymundo', 'apolinar martinez', 'je

### Procesamos todos los textos que faltan

In [ ]:
# Procesamos cada texto extraido de las diferentes imagenes
for ii in range(df.shape[0]):
    print("\n",  df[0][ii])
    # Seleccionamos el texto a procesar
    text = df[1][ii]

    # Extraemos token del texto y corregimos las palabras con el vocabulario
    tokens_corregidos = correction_words(text, vocabulary, tokenizer)

    # Detectamos organizaciones, personas, lugares y funcionarios en el texto extraido
    detect_info = {}

    detect_info['organizations'] = match_info(tokens_corregidos, vectorizer_organizations, bag_of_words_organizations, context_organizations)
    detect_info['prosecuted'] = match_info(tokens_corregidos, vectorizer_prosecuted, bag_of_words_prosecuted, context_prosecuted)
    detect_info['civilservants'] = match_info(tokens_corregidos, vectorizer_civilservants, bag_of_words_civilservants, context_civilservants)
    detect_info['places'] = match_info(tokens_corregidos, vectorizer_places, bag_of_words_places, context_places)

    # Imprimimos los resultados
    for key in detect_info:
        print("--- ", key, ": ")
        print(detect_info[key])



 ../pruebas/images_texto/DFS_Exp._009-011-014,_L-1-5-.JPG.png
---  organizations : 
['grupo constitucion', 'grupo del 68', 'partido comunista', '22 de abril', 'periodico 30-30', 'periodico rojo', 'el mexicano', 'periodico madera', 'el dia', 'diario sol', 'partido comunista mexicano', 'flecha roja', '24 horas', 'el sol', '23 de septiembre', 'fuerzas armadas de liberacion', 'diario el sol de chilpancingo', 'comite de lucha armada', 'diario de toluca', 'escuela preparatorio de chilpancingo, chilpancingo, guerrero', 'policia', 'el sol de chilpancingo', 'liga comunista 23 de septiembre', 'sindicato revolucionario']
---  prosecuted : 
['jose rojo', 'raul sanchez dia', 'juan coro', 'vicente chaparro', 'alfonso tena', 'fernando ante', 'arturo', 'rosalio wences reza', 'mario arturo acosta chaparro escapite', 'juan guillermo hernandez', 'jose toluca', 'maria benitez', 'juan diaz diaz', 'liga comunista 23 de septiembre', 'guillermo hernandez', 'francisco mencio gonzalez', 'mario arturo acosta ch